<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Liberaries

In [21]:
import numpy as np
import pickle
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR,SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

## Reading the prepared dataset and split it

In [22]:
path_local='../../Data/media_with_angles_low_normed.csv'
path_drive='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low_normed.csv'
data =pd.read_csv(path_drive)
data.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,image_name,yaw,pitch,roll
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219,image03195,0.725284,-0.140406,-0.174674
1,0.104239,-0.004250,0.056021,-0.076575,-0.016304,-0.016304,0.019859,-0.582855,0.007804,-0.004250,...,0.030482,-0.680720,-0.656611,-0.632502,-0.801262,-0.825371,image03247,-0.233745,-0.116700,0.095809
2,0.180066,-0.003883,0.180066,0.017758,-0.036345,-0.003883,0.115143,0.028579,0.115143,0.071861,...,0.047943,-0.525545,-0.482263,-0.460622,-0.633751,-0.698674,image03214,-1.330352,0.003072,0.362599
3,0.000114,-0.015663,0.031668,-0.110324,0.000114,0.000114,0.063222,-1.041159,0.078999,0.078999,...,-0.015938,-1.341195,-1.246533,-1.167649,-1.593625,-1.656732,image03238,-0.040907,-0.474087,0.019801
4,-0.107980,-0.005590,-0.056785,-0.069583,0.020008,0.032807,0.058404,-0.799111,0.084002,0.109599,...,0.043323,-0.852588,-0.814192,-0.775796,-0.916582,-0.954978,image03236,0.205732,-0.241359,-0.150232


In [23]:
X = data.iloc[:,:-4]
X.head(1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-0.236824,-0.006193,-0.224011,-0.147134,0.032246,-0.006193,-0.159947,-1.133721,-0.159947,-0.121508,...,0.008491,-0.029947,0.008491,0.021304,0.034117,-0.785904,-0.747465,-0.709027,-0.849968,-0.901219


In [24]:
y = data.iloc[:,-3:]
y.head(1)

,yaw,pitch,roll
0,0.725284,-0.140406,-0.174674


In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

## **Model 1: RandomForest**




In [26]:
clf = MultiOutputRegressor(RandomForestRegressor(max_depth=6, random_state=42, bootstrap=True),n_jobs=-1)
clf.fit(x_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=6,
                                                     random_state=42),
                     n_jobs=-1)

In [27]:
print(f"training score: {clf.score(x_train, y_train, sample_weight=None)}")
print(f"testing score: {clf.score(x_test, y_test, sample_weight=None)}")

training score: 0.8794205922987813
testing score: 0.7221536275744578


In [28]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_Rm.pkl', 'wb') as file:
        pickle.dump(clf, file)

## **Model 2: SVR**
  

In [29]:
# define base model
model = SVR(kernel='rbf', C=300, degree=4,coef0=1) 

# define the direct multioutput wrapper model
wrapper = MultiOutputRegressor(model)
wrapper.fit(x_train, y_train)
y_pred = wrapper.predict(x_test)

In [30]:
print(f"training score: {wrapper.score(x_train, y_train, sample_weight=None)}")
print(f"testing score: {wrapper.score(x_test, y_test, sample_weight=None)}")

training score: 0.7964517949349106
testing score: 0.769823845816001


In [31]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_svr.pkl', 'wb') as file:
        pickle.dump(wrapper, file)

## **Model 3: GradientBoost**



In [32]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42,n_estimators=20,learning_rate=0.05,max_depth=6) #random_state=42,n_estimators=100,learning_rate=0.05
model = MultiOutputRegressor(estimator=gbr,n_jobs=-1)

In [33]:
model.fit(x_train, y_train)
ypred = model.predict(x_test)

In [34]:
print(f"training score: {model.score(x_train, y_train)}")
print(f"testing score: {model.score(x_test, y_test)}")

training score: 0.7905088566135884
testing score: 0.6228090670586276


In [35]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB.pkl', 'wb') as file:
        pickle.dump(model, file)

## **Model 3: XGBoost**



In [36]:
from xgboost.sklearn import XGBRegressor
xbg = XGBRegressor(random_state=42, max_depth=5, n_estimators=300,reg_alpha=0.1)
model = MultiOutputRegressor(estimator=xbg,n_jobs=-1)

In [37]:
model.fit(x_train, y_train)
ypred = model.predict(x_test)

In [38]:
print(f"training score: {model.score(x_train, y_train)}")
print(f"testing score: {model.score(x_test, y_test)}")

training score: 0.9994157640762155
testing score: 0.7258803428019664


In [39]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_XGB.pkl', 'wb') as file:
        pickle.dump(model, file)

#**Single models trial**:

### Yaw prediction

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model_yaw = MultiOutputRegressor(estimator=gbr)
model_yaw.fit(x_train, y_train[['yaw']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [41]:
print(f"training score: {model_yaw.score(x_train, y_train[['yaw']].to_numpy())}")
print(f"testing score: {model_yaw.score(x_test, y_test[['yaw']].to_numpy())}")

training score: 0.9225273908020336
testing score: 0.799039316376515


### Pitch prediction

In [42]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model_pitch = MultiOutputRegressor(estimator=gbr)
model_pitch.fit(x_train, y_train[['pitch']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [43]:
print(f"training score: {model_pitch.score(x_train, y_train[['pitch']].to_numpy())}")
print(f"testing score: {model_pitch.score(x_test, y_test[['pitch']].to_numpy())}")

training score: 0.8440887140474439
testing score: 0.6184808564546651


### Roll prediction

In [44]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42, n_estimators= 50, subsample=1) 
model_roll = MultiOutputRegressor(estimator=gbr)
model_roll.fit(x_train, y_train[['roll']].to_numpy())

MultiOutputRegressor(estimator=GradientBoostingRegressor(n_estimators=50,
                                                         random_state=42,
                                                         subsample=1))

In [45]:
print(f"training score: {model_roll.score(x_train, y_train[['roll']].to_numpy())}")
print(f"testing score: {model_roll.score(x_test, y_test[['roll']].to_numpy())}")

training score: 0.8707538941239709
testing score: 0.7231964095695946


In [46]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_yaw.pkl', 'wb') as file:
        pickle.dump(model_yaw, file)

In [47]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_pitch.pkl', 'wb') as file:
        pickle.dump(model_pitch, file)

In [48]:
# save model
with open('/content/drive/MyDrive/Head Pose Estimation ML Project/Saved Models/model_GB_roll.pkl', 'wb') as file:
        pickle.dump(model_roll, file)